In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

In [48]:
class Pair:
    def __init__(self, asset1, asset2):
        self.asset1 = asset1
        self.asset2 = asset2
        self.df = yf.download([asset1, asset2])

    def backtest(self, start_date, end_date, window, entry_threshold, exit_threshold, verbose=False):
        df = self.df.loc[start_date:end_date]
        # df = df.iloc[::-1]  # reverse data to test bearish conditions
        df = df["Close"]
        df.columns.name = None

        df['ratio'] = df[self.asset1] / df[self.asset2]
        df['ratio_ma'] = df['ratio'].rolling(window=window).mean()
        df['ratio_std'] = df['ratio'].rolling(window=window).std()
        df['z_score'] = (df['ratio'] - df['ratio_ma']) / df['ratio_std']

        df[f'{self.asset1} return'] = df[self.asset1].pct_change()
        df[f'{self.asset2} return'] = df[self.asset2].pct_change()

        df.dropna(inplace=True)

        positions = pd.DataFrame(0, index=df.index, columns=[self.asset1, self.asset2])
        current_position = 0  # 0 for no position, 1 for long-short, -1 for short-long

        # Trading logic based on ratio
        for i in range(len(df)):
            if current_position == 0:
                if df['z_score'].iloc[i] > entry_threshold:
                    positions.iloc[i] = [-1, 1]  # Short etf1, Long etf2
                    current_position = 1
                elif df['z_score'].iloc[i] < -entry_threshold:
                    positions.iloc[i] = [1, -1]  # Long etf1, Short etf2
                    current_position = -1
            else:
                if abs(df['z_score'].iloc[i]) < exit_threshold:
                    positions.iloc[i] = [0, 0]
                    current_position = 0
                else:
                    positions.iloc[i] = positions.iloc[i-1]

        # Shift positions by 1 day for backtesting
        positions = positions.shift(1)
        positions.fillna(0, inplace=True)

        returns = pd.DataFrame(index=df.index)
        returns[self.asset1] = positions[self.asset1] * df[f'{self.asset1} return']
        returns[self.asset2] = positions[self.asset2] * df[f'{self.asset2} return']
        returns['total'] = returns[self.asset1] + returns[self.asset2]

        portfolio_value = (1 + returns['total']).cumprod()

        sharpe_ratio = np.sqrt(252) * returns['total'].mean() / returns['total'].std()
        max_drawdown = (portfolio_value / portfolio_value.cummax() - 1).min()

        # Test for stationarity of ratio
        adf_result = adfuller(df['ratio'].dropna())

        if (verbose):
            # Plots
            fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 12))

            # Plot ratio
            ax1.plot(df['ratio'])
            ax1.set_title(f'Price Ratio ({self.asset1}/{self.asset2})')
            ax1.grid(True)

            # Plot z-score
            ax2.plot(df['z_score'])
            ax2.axhline(y=entry_threshold, color='r', linestyle='--')
            ax2.axhline(y=-entry_threshold, color='r', linestyle='--')
            ax2.axhline(y=0, color='black', linestyle='-')
            ax2.set_title('Z-Score of Ratio')
            ax2.grid(True)

            # Plot positions
            ax3.plot(positions[self.asset1], label=f'{self.asset1} position')
            ax3.plot(positions[self.asset2], label=f'{self.asset2} position')
            ax3.set_title('Positions')
            ax3.legend()
            ax3.grid(True)

            # Plot portfolio value
            ax4.plot(portfolio_value)
            ax4.set_title('Portfolio Value')
            ax4.grid(True)

            plt.tight_layout()
            plt.show()

            print(f"ADF test p-value for ratio: {adf_result[1]:.4f}")
            print(f"Performance Metrics:")
            print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
            print(f"Maximum Drawdown: {max_drawdown:.2%}")
            print(f"Final Portfolio Value: {portfolio_value.iloc[-1]:.4f}")

            # Print some trading statistics
            trade_changes = (positions != positions.shift(1)).any(axis=1)
            num_trades = trade_changes.sum() / 2  # Divide by 2 since each trade involves both ETFs
            print(f"\nNumber of trades: {num_trades:.0f}")
            print(f"Average holding period: {len(df)/num_trades:.1f} days")

            # Print some position statistics
            print("\nPosition Statistics:")
            print(f"Days with positions: {(positions != 0).any(axis=1).sum()}")
            print(f"Days with no positions: {(positions == 0).all(axis=1).sum()}")

        return [portfolio_value.iloc[-1], sharpe_ratio, max_drawdown]

In [49]:
# IBIT, ARKB, BITB, GBTC, FBTC, BTC
pair = Pair(asset1="BTC-EUR", asset2="BTC-GBP")

[*********************100%***********************]  2 of 2 completed


Reversed dataset to test on bearish market

Backtest with cross validation to prevent overfitting parameters to this one specific dataset

Implement adaptive parameters - have another larger rolling window to change parameters on the fly

Use RL

In [50]:
results = pd.DataFrame(columns=["window", "entry_threshold", "exit_threshold", "total_return", "sharpe_ratio", "max_drawdown"])

for window in [5, 7, 10, 15]:
    for entry_threshold in [0.5, 1, 1.2]:
        for exit_threshold in [0.3, 0.5, 1]:
            result = pair.backtest(
                start_date="2024-2-21", 
                end_date="2025-02-21", 
                window=window, 
                entry_threshold=entry_threshold, 
                exit_threshold=exit_threshold,
                verbose=False
            )
            results.loc[len(results)] = [window, entry_threshold, exit_threshold, *result]

results

,window,entry_threshold,exit_threshold,total_return,sharpe_ratio,max_drawdown
0,5.0,0.5,0.3,1.032241,0.683838,-0.030036
1,5.0,0.5,0.5,1.036347,0.788338,-0.031488
2,5.0,0.5,1.0,1.021061,0.494638,-0.024029
3,5.0,1.0,0.3,1.051550,1.105994,-0.029672
4,5.0,1.0,0.5,1.032944,0.735266,-0.031758
5,5.0,1.0,1.0,1.019350,0.512540,-0.020172
6,5.0,1.2,0.3,1.054472,1.167546,-0.029672
7,5.0,1.2,0.5,1.038150,0.851669,-0.030030
8,5.0,1.2,1.0,1.036386,0.979643,-0.019179
9,7.0,0.5,0.3,0.987809,-0.239112,-0.034703


In [ ]:
import torch
import torch_directml

In [9]:
dml = torch_directml.device()
tensor1 = torch.tensor([1]).to(dml) # Note that dml is a variable, not a string!
tensor2 = torch.tensor([2]).to(dml)
dml_algebra = tensor1 + tensor2
dml_algebra.item()

3